In [ ]:
# !pip install matplotlib

In [12]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import Dataset,DataLoader  # Gives easier dataset managment and creates mini batches
from torchvision.transforms.functional import InterpolationMode
from skimage import exposure, img_as_ubyte
from PIL import Image

In [13]:
def noisify(dataset='mnist', nb_classes=10, train_labels=None, noise_type=None, noise_rate=0, random_state=0):
    if noise_type == 'pairflip':
        train_noisy_labels, actual_noise_rate = noisify_pairflip(train_labels, noise_rate, random_state=random_state, nb_classes=nb_classes)
    if noise_type == 'symmetric':
        train_noisy_labels, actual_noise_rate = noisify_multiclass_symmetric(train_labels, noise_rate, random_state=random_state, nb_classes=nb_classes)
    if noise_type == 'asymmetric':
        if dataset == 'mnist':
            train_noisy_labels, actual_noise_rate = noisify_mnist_asymmetric(train_labels, noise_rate, random_state=random_state)
        elif dataset == 'cifar10':
            train_noisy_labels, actual_noise_rate = noisify_cifar10_asymmetric(train_labels, noise_rate, random_state=random_state)
        elif dataset == 'cifar100':
            train_noisy_labels, actual_noise_rate = noisify_cifar100_asymmetric(train_labels, noise_rate, random_state=random_state)
    return train_noisy_labels, actual_noise_rate

In [14]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

#the variables we have modified
noise_or_not = False
batch_size = 5

class JoCoR:
    def __init__(self, args, train_dataset, device, input_channel, num_classes):

        # Hyper Parameters
        self.batch_size = batch_size
        
        learning_rate = args.lr

        if args.forget_rate is None:
            if args.noise_type == "asymmetric":
                forget_rate = args.noise_rate / 2
            else:
                forget_rate = args.noise_rate
        else:
            forget_rate = args.forget_rate

        self.noise_or_not = noise_or_not  #train_dataset.noise_or_not

        # Adjust learning rate and betas for Adam Optimizer
        mom1 = 0.9
        mom2 = 0.1
        self.alpha_plan = [learning_rate] * args.n_epoch
        self.beta1_plan = [mom1] * args.n_epoch

        for i in range(args.epoch_decay_start, args.n_epoch):
            self.alpha_plan[i] = float(args.n_epoch - i) / (args.n_epoch - args.epoch_decay_start) * learning_rate
            self.beta1_plan[i] = mom2

        # define drop rate schedule
        self.rate_schedule = np.ones(args.n_epoch) * forget_rate
#         print(np.linspace(0, forget_rate ** args.exponent, args.num_gradual).shape())
        self.rate_schedule[:args.num_gradual] = np.linspace(0, forget_rate ** args.exponent, args.num_gradual)

        self.device = device
        self.num_iter_per_epoch = args.num_iter_per_epoch
        self.print_freq = args.print_freq
        self.co_lambda = args.co_lambda
        self.n_epoch = args.n_epoch
        self.train_dataset = train_dataset

        if args.model_type == "cnn":
            self.model1 = CNN(input_channel=input_channel, n_outputs=num_classes)
            self.model2 = CNN(input_channel=input_channel, n_outputs=num_classes)
        elif args.model_type == "mlp":
            self.model1 = MLPNet()
            self.model2 = MLPNet()

        self.model1.to(device)
        print(self.model1.parameters)

        self.model2.to(device)
        print(self.model2.parameters)

        self.optimizer = torch.optim.Adam(list(self.model1.parameters()) + list(self.model2.parameters()),
                                          lr=learning_rate)

        self.loss_fn = loss_jocor


        self.adjust_lr = args.adjust_lr

    # Evaluate the Model
    def evaluate(self, test_loader):
        print('Evaluating ...')
        print(f'the shape of the testloader {len(test_loader)}')
        self.model1.eval()  # Change model to 'eval' mode.
        self.model2.eval()  # Change model to 'eval' mode

        correct1 = 0
        total1 = 0
        for images, labels, _ in test_loader:
            
            print(f' shape of images is {images.size()}')
            images = Variable(images).to(self.device)
            logits1 = self.model1(images)
            print(f' logits1 {logits1.size()}')
            
            outputs1 = F.softmax(logits1, dim=1)
            print(f' outout1 {outputs1.size()}')
            _, pred1 = torch.max(outputs1.data, 1)
            print(f' shape of pred1 is {pred1.size()}')
            total1 += labels.size(0)
            
            print(f' shape of pred1 is {pred1.size()}')
            print(f' shape of labels is {labels.size()}')

            correct1 += (pred1.cpu() == labels).sum()

        correct2 = 0
        total2 = 0
        for images, labels, _ in test_loader:
            images = Variable(images).to(self.device)
            logits2 = self.model2(images)
            outputs2 = F.softmax(logits2, dim=1)
            _, pred2 = torch.max(outputs2.data, 1)
            total2 += labels.size(0)
            correct2 += (pred2.cpu() == labels).sum()

        acc1 = 100 * float(correct1) / float(total1)
        acc2 = 100 * float(correct2) / float(total2)
        return acc1, acc2

    # Train the Model
    def train(self, train_loader, epoch):
        print('Training ...')
        self.model1.train()  # Change model to 'rain' mode.
        self.model2.train()  # Change model to 'train' mode

        if self.adjust_lr == 1:
            self.adjust_learning_rate(self.optimizer, epoch)

        train_total = 0
        train_correct = 0
        train_total2 = 0
        train_correct2 = 0
        pure_ratio_1_list = []
        pure_ratio_2_list = []

        for i, (images, labels, index) in enumerate(train_loader):
            ind = indexes.cpu().numpy().transpose()
            if i > self.num_iter_per_epoch:
                break

            images = Variable(images).to(self.device)
            labels = Variable(labels).to(self.device)

            # Forward + Backward + Optimize
            logits1 = self.model1(images)
            prec1 = accuracy(logits1, labels, topk=(1,))
            train_total += 1
            train_correct += prec1

            logits2 = self.model2(images)
            prec2 = accuracy(logits2, labels, topk=(1,))
            train_total2 += 1
            train_correct2 += prec2

            loss_1, loss_2, pure_ratio_1, pure_ratio_2 = self.loss_fn(logits1, logits2, labels, self.rate_schedule[epoch],
                                                                 ind, self.noise_or_not, self.co_lambda)

            self.optimizer.zero_grad()
            loss_1.backward()
            self.optimizer.step()

            pure_ratio_1_list.append(100 * pure_ratio_1)
            pure_ratio_2_list.append(100 * pure_ratio_2)

            if (i + 1) % self.print_freq == 0:
                print(
                    'Epoch [%d/%d], Iter [%d/%d] Training Accuracy1: %.4F, Training Accuracy2: %.4f, Loss1: %.4f, Loss2: %.4f, Pure Ratio1 %.4f %% Pure Ratio2 %.4f %%'
                    % (epoch + 1, self.n_epoch, i + 1, len(self.train_dataset) // self.batch_size, prec1, prec2,
                       loss_1.data.item(), loss_2.data.item(), sum(pure_ratio_1_list) / len(pure_ratio_1_list), sum(pure_ratio_2_list) / len(pure_ratio_2_list)))

        train_acc1 = float(train_correct) / float(train_total)
        train_acc2 = float(train_correct2) / float(train_total2)
        return train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list

    def adjust_learning_rate(self, optimizer, epoch):
        for param_group in optimizer.param_groups:
            param_group['lr'] = self.alpha_plan[epoch]
            param_group['betas'] = (self.beta1_plan[epoch], 0.999)  # Only change beta1

In [16]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init 
import torch.nn.functional as F
import torch.optim as optim

def call_bn(bn, x):
    return bn(x)

#number of classes
img_resolution= 512*682    #512*682
# num_classes = n_outputs

class CNN(nn.Module):
    def __init__(self, input_channel=3, n_outputs=num_classes, dropout_rate=0.25, momentum=0.1):
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        super(CNN, self).__init__()
        self.c1=nn.Conv2d(input_channel, 64,kernel_size=3,stride=1, padding=1)
        self.c2=nn.Conv2d(64,64,kernel_size=3,stride=1, padding=1)
        self.c3=nn.Conv2d(64,128,kernel_size=3,stride=1, padding=1)
        self.c4=nn.Conv2d(128,128,kernel_size=3,stride=1, padding=1)
        self.c5=nn.Conv2d(128,196,kernel_size=3,stride=1, padding=1)
        self.c6=nn.Conv2d(196,16,kernel_size=3,stride=1, padding=1)
        self.linear1=nn.Linear(256, n_outputs)
        self.bn1=nn.BatchNorm2d(64, momentum=self.momentum)
        self.bn2=nn.BatchNorm2d(64, momentum=self.momentum)
        self.bn3=nn.BatchNorm2d(128, momentum=self.momentum)
        self.bn4=nn.BatchNorm2d(128, momentum=self.momentum)
        self.bn5=nn.BatchNorm2d(196, momentum=self.momentum)
        self.bn6=nn.BatchNorm2d(16, momentum=self.momentum)

    def forward(self, x,):
        h=x
        h=self.c1(h)
        h=F.relu(call_bn(self.bn1, h))
        h=self.c2(h)
        h=F.relu(call_bn(self.bn2, h))
        h=F.max_pool2d(h, kernel_size=2, stride=2)

        h=self.c3(h)
        h=F.relu(call_bn(self.bn3, h))
        h=self.c4(h)
        h=F.relu(call_bn(self.bn4, h))
        h=F.max_pool2d(h, kernel_size=2, stride=2)

        h=self.c5(h)
        h=F.relu(call_bn(self.bn5, h))
        h=self.c6(h)
        h=F.relu(call_bn(self.bn6, h))
        h=F.max_pool2d(h, kernel_size=2, stride=2)

        h = h.view(h.size(0), -1)
        logit=self.linear1(h)
        return logit

class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.layers = nn.Sequential(self.fc1 == nn.Linear(1536*2048*3, 256),self.fc2 == nn.Linear(256, n_outputs))    

    def forward(self, x):
#         x = x.view(-1, img_resolution)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
        return self.layers(x)

NameError: name 'n_outputs' is not defined

In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F



def plot_result(accuracy_list,pure_ratio_list,name="test.png"):
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    plt.plot(accuracy_list, label='test_accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(pure_ratio_list, label='test_pure_ratio')
    plt.savefig(name)


def accuracy(logit, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    output = F.softmax(logit, dim=1)
    maxk = max(topk)
    batch_size = batch_size

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res[0]

In [11]:
def kl_loss_compute(pred, soft_targets, reduce=True):

    kl = F.kl_div(F.log_softmax(pred, dim=1),F.softmax(soft_targets, dim=1),reduce=False)

    if reduce:
        return torch.mean(torch.sum(kl, dim=1))
    else:
        return torch.sum(kl, 1)




def loss_jocor(y_1, y_2, t, forget_rate, ind, noise_or_not, co_lambda=0.1):

    loss_pick_1 = F.cross_entropy(y_1, t, reduce = False) * (1-co_lambda)
    loss_pick_2 = F.cross_entropy(y_2, t, reduce = False) * (1-co_lambda)
    loss_pick = (loss_pick_1 + loss_pick_2 + co_lambda * kl_loss_compute(y_1, y_2,reduce=False) + co_lambda * kl_loss_compute(y_2, y_1, reduce=False)).cpu()


    ind_sorted = np.argsort(loss_pick.data)
    loss_sorted = loss_pick[ind_sorted]

    remember_rate = 1 - forget_rate
    num_remember = int(remember_rate * len(loss_sorted))

    pure_ratio = np.sum(noise_or_not[ind[ind_sorted[:num_remember]]])/float(num_remember)

    ind_update=ind_sorted[:num_remember]

    # exchange
    loss = torch.mean(loss_pick[ind_update])

    return loss, loss, pure_ratio, pure_ratio

In [12]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

#the variables we have modified
noise_or_not = False
batch_size = 5

class JoCoR:
    def __init__(self, args, train_dataset, device, input_channel, num_classes):

        # Hyper Parameters
        self.batch_size = batch_size
        
        learning_rate = args.lr

        if args.forget_rate is None:
            if args.noise_type == "asymmetric":
                forget_rate = args.noise_rate / 2
            else:
                forget_rate = args.noise_rate
        else:
            forget_rate = args.forget_rate

        self.noise_or_not = noise_or_not  #train_dataset.noise_or_not

        # Adjust learning rate and betas for Adam Optimizer
        mom1 = 0.9
        mom2 = 0.1
        self.alpha_plan = [learning_rate] * args.n_epoch
        self.beta1_plan = [mom1] * args.n_epoch

        for i in range(args.epoch_decay_start, args.n_epoch):
            self.alpha_plan[i] = float(args.n_epoch - i) / (args.n_epoch - args.epoch_decay_start) * learning_rate
            self.beta1_plan[i] = mom2

        # define drop rate schedule
        self.rate_schedule = np.ones(args.n_epoch) * forget_rate
#         print(np.linspace(0, forget_rate ** args.exponent, args.num_gradual).shape())
        self.rate_schedule[:args.num_gradual] = np.linspace(0, forget_rate ** args.exponent, args.num_gradual)

        self.device = device
        self.num_iter_per_epoch = args.num_iter_per_epoch
        self.print_freq = args.print_freq
        self.co_lambda = args.co_lambda
        self.n_epoch = args.n_epoch
        self.train_dataset = train_dataset

        if args.model_type == "cnn":
            self.model1 = CNN(input_channel=input_channel, n_outputs=num_classes)
            self.model2 = CNN(input_channel=input_channel, n_outputs=num_classes)
        elif args.model_type == "mlp":
            self.model1 = MLPNet()
            self.model2 = MLPNet()

        self.model1.to(device)
        print(self.model1.parameters)

        self.model2.to(device)
        print(self.model2.parameters)

        self.optimizer = torch.optim.Adam(list(self.model1.parameters()) + list(self.model2.parameters()),
                                          lr=learning_rate)

        self.loss_fn = loss_jocor


        self.adjust_lr = args.adjust_lr

    # Evaluate the Model
    def evaluate(self, test_loader):
        print('Evaluating ...')
        print(f'the shape of the testloader {len(test_loader)}')
        self.model1.eval()  # Change model to 'eval' mode.
        self.model2.eval()  # Change model to 'eval' mode

        correct1 = 0
        total1 = 0
        for images, labels, _ in test_loader:
            
            print(f' shape of images is {images.size()}')
            images = Variable(images).to(self.device)
            logits1 = self.model1(images)
            print(f' logits1 {logits1.size()}')
            
            outputs1 = F.softmax(logits1, dim=1)
            print(f' outout1 {outputs1.size()}')
            _, pred1 = torch.max(outputs1.data, 1)
            print(f' shape of pred1 is {pred1.size()}')
            total1 += labels.size(0)
            
            print(f' shape of pred1 is {pred1.size()}')
            print(f' shape of labels is {labels.size()}')

            correct1 += (pred1.cpu() == labels).sum()

        correct2 = 0
        total2 = 0
        for images, labels, _ in test_loader:
            images = Variable(images).to(self.device)
            logits2 = self.model2(images)
            outputs2 = F.softmax(logits2, dim=1)
            _, pred2 = torch.max(outputs2.data, 1)
            total2 += labels.size(0)
            correct2 += (pred2.cpu() == labels).sum()

        acc1 = 100 * float(correct1) / float(total1)
        acc2 = 100 * float(correct2) / float(total2)
        return acc1, acc2

    # Train the Model
    def train(self, train_loader, epoch):
        print('Training ...')
        self.model1.train()  # Change model to 'rain' mode.
        self.model2.train()  # Change model to 'train' mode

        if self.adjust_lr == 1:
            self.adjust_learning_rate(self.optimizer, epoch)

        train_total = 0
        train_correct = 0
        train_total2 = 0
        train_correct2 = 0
        pure_ratio_1_list = []
        pure_ratio_2_list = []

        for i, (images, labels, index) in enumerate(train_loader):
            ind = indexes.cpu().numpy().transpose()
            if i > self.num_iter_per_epoch:
                break

            images = Variable(images).to(self.device)
            labels = Variable(labels).to(self.device)

            # Forward + Backward + Optimize
            logits1 = self.model1(images)
            prec1 = accuracy(logits1, labels, topk=(1,))
            train_total += 1
            train_correct += prec1

            logits2 = self.model2(images)
            prec2 = accuracy(logits2, labels, topk=(1,))
            train_total2 += 1
            train_correct2 += prec2

            loss_1, loss_2, pure_ratio_1, pure_ratio_2 = self.loss_fn(logits1, logits2, labels, self.rate_schedule[epoch],
                                                                 ind, self.noise_or_not, self.co_lambda)

            self.optimizer.zero_grad()
            loss_1.backward()
            self.optimizer.step()

            pure_ratio_1_list.append(100 * pure_ratio_1)
            pure_ratio_2_list.append(100 * pure_ratio_2)

            if (i + 1) % self.print_freq == 0:
                print(
                    'Epoch [%d/%d], Iter [%d/%d] Training Accuracy1: %.4F, Training Accuracy2: %.4f, Loss1: %.4f, Loss2: %.4f, Pure Ratio1 %.4f %% Pure Ratio2 %.4f %%'
                    % (epoch + 1, self.n_epoch, i + 1, len(self.train_dataset) // self.batch_size, prec1, prec2,
                       loss_1.data.item(), loss_2.data.item(), sum(pure_ratio_1_list) / len(pure_ratio_1_list), sum(pure_ratio_2_list) / len(pure_ratio_2_list)))

        train_acc1 = float(train_correct) / float(train_total)
        train_acc2 = float(train_correct2) / float(train_total2)
        return train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list

    def adjust_learning_rate(self, optimizer, epoch):
        for param_group in optimizer.param_groups:
            param_group['lr'] = self.alpha_plan[epoch]
            param_group['betas'] = (self.beta1_plan[epoch], 0.999)  # Only change beta1

In [13]:
# pwd

In [14]:
root_dir = '/home/subrat/JoCoR_bach/Data/train_image/'
csv_file = 'Clean_train_data_encd.csv'
class BACH(Dataset):
    def __init__(self, 
                 csv_file="/home/subrat/JoCoR_bach/Data/Clean_train_data_encd.csv", 
                 train=True,
                 transform=None, target_transform=None,
                 download=False,):
        self.csv_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv_data)

    def __getitem__(self, index):
        label=self.csv_data.loc[index, 'label']
        img_path = self.csv_data.loc[index, 'Name']

        img_path = os.path.join(root_dir,img_path)
        #print(img_path)
        image = io.imread(img_path)
#         print(image)
#         image = Image.open(img_path).convert("RGB")

        image = img_as_ubyte(exposure.rescale_intensity(image))
#         print(image)
#         exit()
        
        #plt.imshow(image, cmap=None)
#         transform = transforms.Resize((300,350))
#         resized_img = transform(image)
#         print(resized_img.shape)
#         y_label = torch.tensor(int(self.annotations.iloc[index, 2]))

        if self.transform:
            image = self.transform(image)

        return (image, label, index)


In [15]:
# -*- coding:utf-8 -*-
import os
import torch
import torchvision.transforms as transforms
# from JoCoR.data.cifar import CIFAR10, CIFAR100
# from JoCoR.mnist_HAM import HAM10000
import argparse, sys
import datetime





parser = argparse.ArgumentParser()
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--result_dir', type=str, help='dir to save result txt files', default='results')
parser.add_argument('--noise_rate', type=float, help='corruption rate, should be less than 1', default=0.2)
parser.add_argument('--forget_rate', type=float, help='forget rate', default=None)
parser.add_argument('--noise_type', type=str, help='[pairflip, symmetric]', default='pairflip')
parser.add_argument('--num_gradual', type=int, default=2,
                    help='how many epochs for linear drop rate, can be 5, 10, 15. This parameter is equal to Tk for R(T) in Co-teaching paper.')
parser.add_argument('--exponent', type=float, default=1,
                    help='exponent of the forget rate, can be 0.5, 1, 2. This parameter is equal to c in Tc for R(T) in Co-teaching paper.')
parser.add_argument('--dataset', type=str, help='mnist, cifar10, or cifar100', default='bach')
parser.add_argument('--n_epoch', type=int, default=200)
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--print_freq', type=int, default=50)
parser.add_argument('--num_workers', type=int, default=4, help='how many subprocesses to use for data loading')
parser.add_argument('--num_iter_per_epoch', type=int, default=400)
parser.add_argument('--epoch_decay_start', type=int, default=80)
parser.add_argument('--gpu', type=int, default=None)
parser.add_argument('--co_lambda', type=float, default=0.1)
parser.add_argument('--adjust_lr', type=int, default=1)
parser.add_argument('--model_type', type=str, help='[mlp,cnn]', default='cnn')
parser.add_argument('--save_model', type=str, help='save model?', default="False")
parser.add_argument('--save_result', type=str, help='save result?', default="True")



args = parser.parse_args(args=[])

# Seed
torch.manual_seed(args.seed)
if args.gpu is not None:
    device = torch.device('cuda:{}'.format(args.gpu))
    torch.cuda.manual_seed(args.seed)

else:
    device = torch.device('cpu')
    torch.manual_seed(args.seed)

# Hyper Parameters
batch_size = batch_size
learning_rate = args.lr


#load dataset
if args.dataset == 'bach':
    input_channel = 3
    num_classes = 4
    init_epoch = 20
    args.epoch_decay_start = 80
    filter_outlier = True
    args.model_type = "cnn"
    # args.n_epoch = 200
    
    transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
    train_dataset = BACH(download=False,
                            train=True,
                            transform=transform
                            )

    test_dataset = BACH(download=False,
                           train=False,
                           transform=transform
                           )
    
    

    
    
    
    
def main():
    # Data Loader (Input Pipeline)
    print('loading dataset...')
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               num_workers=args.num_workers,
                                               drop_last=True,
                                               shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              num_workers=args.num_workers,
                                              drop_last=True,
                                              shuffle=False)
    # Define models
    print('building model...')

    model = JoCoR(args, train_dataset, device, input_channel, num_classes)

    epoch = 0
    train_acc1 = 0
    train_acc2 = 0

    # evaluate models with random weights
    test_acc1, test_acc2 = model.evaluate(test_loader)

    print(
        'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f ' % (
            epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2))


    acc_list = []
    # training
    for epoch in range(1, args.n_epoch):
        # train models
        train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list = model.train(train_loader, epoch)

        # evaluate models
        test_acc1, test_acc2 = model.evaluate(test_loader)

        # save results
        if pure_ratio_1_list is None or len(pure_ratio_1_list) == 0:
            print(
                'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f' % (
                    epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2))
        else:
            # save results
            mean_pure_ratio1 = sum(pure_ratio_1_list) / len(pure_ratio_1_list)
            mean_pure_ratio2 = sum(pure_ratio_2_list) / len(pure_ratio_2_list)
            print(
                'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %%, Pure Ratio 1 %.4f %%, Pure Ratio 2 %.4f %%' % (
                    epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2, mean_pure_ratio1,
                    mean_pure_ratio2))


        if epoch >= 190:
            acc_list.extend([test_acc1, test_acc2])

    avg_acc = sum(acc_list)/len(acc_list)
    print(len(acc_list))
    print("the average acc in last 10 epochs: {}".format(str(avg_acc)))


In [16]:
# !nvidia-smi


In [17]:
if __name__ == '__main__':
    main()

loading dataset...
building model...


NameError: name 'CNN' is not defined

In [ ]:
# # -*- coding:utf-8 -*-
# import os
# import torch
# import torchvision.transforms as transforms
# # from JoCoR.data.cifar import CIFAR10, CIFAR100
# # from JoCoR.mnist_HAM import HAM10000
# import argparse, sys
# import datetime





# parser = argparse.ArgumentParser()
# parser.add_argument('--lr', type=float, default=0.001)
# parser.add_argument('--result_dir', type=str, help='dir to save result txt files', default='results')
# parser.add_argument('--noise_rate', type=float, help='corruption rate, should be less than 1', default=0.2)
# parser.add_argument('--forget_rate', type=float, help='forget rate', default=None)
# parser.add_argument('--noise_type', type=str, help='[pairflip, symmetric]', default='pairflip')
# parser.add_argument('--num_gradual', type=int, default=2,
#                     help='how many epochs for linear drop rate, can be 5, 10, 15. This parameter is equal to Tk for R(T) in Co-teaching paper.')
# parser.add_argument('--exponent', type=float, default=1,
#                     help='exponent of the forget rate, can be 0.5, 1, 2. This parameter is equal to c in Tc for R(T) in Co-teaching paper.')
# parser.add_argument('--dataset', type=str, help='mnist, cifar10, or cifar100', default='bach')
# parser.add_argument('--n_epoch', type=int, default=200)
# parser.add_argument('--seed', type=int, default=1)
# parser.add_argument('--print_freq', type=int, default=50)
# parser.add_argument('--num_workers', type=int, default=4, help='how many subprocesses to use for data loading')
# parser.add_argument('--num_iter_per_epoch', type=int, default=400)
# parser.add_argument('--epoch_decay_start', type=int, default=80)
# parser.add_argument('--gpu', type=int, default=None)
# parser.add_argument('--co_lambda', type=float, default=0.1)
# parser.add_argument('--adjust_lr', type=int, default=1)
# parser.add_argument('--model_type', type=str, help='[mlp,cnn]', default='cnn')
# parser.add_argument('--save_model', type=str, help='save model?', default="False")
# parser.add_argument('--save_result', type=str, help='save result?', default="True")



# args = parser.parse_args(args=[])

# # Seed
# torch.manual_seed(args.seed)
# if args.gpu is not None:
#     device = torch.device('cuda:{}'.format(args.gpu))
#     torch.cuda.manual_seed(args.seed)

# else:
#     device = torch.device('cpu')
#     torch.manual_seed(args.seed)

# # Hyper Parameters
# batch_size = batch_size
# learning_rate = args.lr

# # load dataset
# # if args.dataset == 'HAM10000':
# #     input_channel = 3
# #     num_classes = 7
# #     init_epoch = 10
# #     filter_outlier = True
# #     args.epoch_decay_start = 40
# #     args.model_type = "mlp"
# #     args.n_epoch = 200
# #     dataset = HAM10000(root='./home/subrat/JoCoR-env/archive/HAM10000_images_part_1/',
# #                           train=True,
# #                           transform=transforms.ToTensor(),
# #                           noise_type=args.noise_type,
# #                           noise_rate=args.noise_rate
# #                           )
# #     print(dataset)
# #     test_dataset = HAM10000(root='/home/subrat/JoCoR-env/archive/HAM10000_images_part_1/',
# #                          train=False,


# if args.forget_rate is None:
#     forget_rate = args.noise_rate
# else:
#     forget_rate = args.forget_rate

    
# # input_channel = 3
# # num_classes = 7
# # init_epoch = 10
# # filter_outlier = True
# # args.epoch_decay_start = 40
# # args.model_type = "mlp"
# # args.n_epoch = 200
# # dataset = HAM10000(root='./home/subrat/JoCoR-env/archive/HAM10000_images_part_1/',
# #                       train=True,
# #                       transform=transforms.ToTensor(),
# #                       noise_type=args.noise_type,
# #                       noise_rate=args.noise_rate
# #                       )
# # Load Data
# # transform for rectangular resize

# transform=transforms.Compose([
#         transforms.ToPILImage(),
#         transforms.Resize(512, interpolation=InterpolationMode.BILINEAR, max_size=None, antialias=None),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor()
#     ])
# # img = Image.fromarray(np.astype(np.uint8))


# # load dataset
# if args.dataset == 'bach':
#     input_channel = 3
#     num_classes = 4
#     init_epoch = 10
#     filter_outlier = True
#     args.epoch_decay_start = 40
#     args.model_type = "cnn"
#     args.n_epoch = 2



# dataset = BACH(transform=transform)

    
# train_percentage = 0.4
# val_percentage = 0.3
# train_size = int(train_percentage * len(dataset))
# val_size = int(val_percentage * len(dataset))
# test_size = len(dataset) - train_size - val_size


# def main():
#     # Data Loader (Input Pipeline)
# #     print('loading dataset...')
    

#     train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
#     print('dataset is loaded')
#     # train_set, test_set = torch.utils.data.random_split(dataset, train_set[0],test_set[0])
#     train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
#     val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
#     test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=2)

#     # Define models
#     print('building model...')

#     model = JoCoR(args, train_dataset, device, input_channel, num_classes)
#     print(model)

#     epoch = 0
#     train_acc1 = 0
#     train_acc2 = 0

#     # evaluate models with random weights
#     test_acc1, test_acc2 = model.evaluate(test_loader)

#     print(
#         'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f ' % (
#             epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2))

# #     print(test_loader.size())
#     acc_list = []
#     # training
#     for epoch in range(1, args.n_epoch):
#         # train models
#         train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list = model.train(train_loader, epoch)

#         # evaluate models
#         test_acc1, test_acc2 = model.evaluate(test_loader)

#         # save results
#         if pure_ratio_1_list is None or len(pure_ratio_1_list) == 0:
#             print(
#                 'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f' % (
#                     epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2))
#         else:
#             # save results
#             mean_pure_ratio1 = sum(pure_ratio_1_list) / len(pure_ratio_1_list)
#             mean_pure_ratio2 = sum(pure_ratio_2_list) / len(pure_ratio_2_list)
#             print(
#                 'Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %%, Pure Ratio 1 %.4f %%, Pure Ratio 2 %.4f %%' % (
#                     epoch + 1, args.n_epoch, len(test_dataset), test_acc1, test_acc2, mean_pure_ratio1,
#                     mean_pure_ratio2))


#         if epoch >= 190:
#             acc_list.extend([test_acc1,test_acc2])

#     avg_acc = sum(acc_list)/len(acc_list)
#     print(len(acc_list))
#     print("the average acc in last 10 epochs: {}".format(str(avg_acc)))




In [ ]:
# if __name__ == '__main__':
#     main()

In [ ]:
# ls /home/subrat/